### Interactive Cropping Tool Demo

In [4]:
from crop import interactivecrop
from crop.samples import sample_images, sample_names

interactivecrop.main(sample_images,image_name_list = sample_names, optimize=True,crop_shape= 'Ellipse')

GridBox(children=(Dropdown(description='Img Name:', layout=Layout(grid_area='im_selector', width='100%'), opti…

In [ ]:
from in